In [ ]:
import datetime 

In [ ]:
## Start Dates

start_date = datetime.date(2010, 1, 1)
end_date = datetime.date(2020, 12, 1)
delta = datetime.timedelta(days=31)


start_list =[]

while start_date <= end_date:
    start_list.append(start_date)
    start_date += delta
    
start_list

In [ ]:
## End Dates

start_date = datetime.date(2010, 1, 31)
end_date = datetime.date(2020, 12, 31)
delta = datetime.timedelta(days=31)


end_list =[]

while start_date <= end_date:
    end_list.append(start_date)
    start_date += delta
    
end_list

In [ ]:
##string conversion
# date_range = ['20100101:20100131']

end_list.strftime("%Y%m%d")
end_list

In [ ]:
#converting lists to strings 

end_str_list = []
for i in range(0,len(end_list)):
    end_str_list.append(end_list[i].strftime('%Y%m%d'))
    
start_str_list =[]
for i in range(0,len(start_list)):
    start_str_list.append(start_list[i].strftime('%Y%m%d'))
    
start_str_list

In [ ]:
#list of endpoints for API
complete_endpoints = []
for i in range(0,len(start_str_list)):
    complete_endpoints.append(f"{start_str_list[i]}:{end_str_list[i]}")

complete_endpoints